In [1]:
!sudo apt-get install -y build-essential libasound2-dev libjack-dev portaudio19-dev
!pip install magenta --ignore-installed llvmlite

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libjack0 libportaudio2 libportaudiocpp0 uuid-dev
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev uuid-dev
0 upgraded, 7 newly installed, 1 to remove and 76 not upgraded.
Need to get 625 kB of archives.
After this operation, 3340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasound2-dev amd64 1.2.2-2.1ubuntu2.5 [104 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 uuid-dev amd64 2.34-0.1ubuntu9.

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import logging
import sys
tf.get_logger().setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
tf.get_logger().addHandler(handler)
tf.debugging.set_log_device_placement(True)

from magenta.scripts.convert_dir_to_note_sequences import convert_musicxml
from note_seq import split_note_sequence
import tensorflow.compat.v1 as tf
import os
prepath = "/kaggle/input/bachxml/xml"
all_ns = []
count = 0
for file in os.listdir(prepath):
    try:
        ns = convert_musicxml(prepath, "xml", os.path.join(prepath, file))

    except: 
        print("Error on:", file)
        count += 1
        continue
    
    ns_split = split_note_sequence(ns,
                            20,
                            False)
    count += 1
    all_ns += ns_split
    if count%100 == 0:
        print(count)

print(len(all_ns))
!mkdir /kaggle/tmp/
if (os.path.exists("/kaggle/tmp/bach.tfrecord") == False):
    f = open("/kaggle/tmp/bach.tfrecord", "w")
with tf.io.TFRecordWriter("/kaggle/tmp/bach.tfrecord") as writer:
    for sequence in all_ns:
        writer.write(sequence.SerializeToString())

2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,211 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/013906B_.xml.
2023-04-16 18:36:33,233 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/037000B_.xml.
2023-04-16 18:36:33,233 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/037000B_.xml.
2023-04-16 18:36:33,233 - tensorflow - INFO - Converted MusicXML file /kaggle/input/bachxml/xml/037000B_.xml.
2023-04-16

In [11]:
from magenta.pipelines import pipeline, note_sequence_pipelines, statistics
from note_seq.protobuf import music_pb2
from magenta.models import polyphony_rnn
from magenta.models.polyphony_rnn import polyphony_rnn_pipeline
from magenta.models.polyphony_rnn import polyphony_model
pipeline_instance = polyphony_rnn.polyphony_rnn_pipeline.get_pipeline(
    min_steps=80,  # 5 measures
    max_steps = None,
    eval_ratio=0.2,
    config=polyphony_model.default_configs['polyphony'])

input_dir = '/kaggle/tmp/bach.tfrecord'
output_dir = '/kaggle/working/split'
pipeline.run_pipeline_serial(
  pipeline_instance,
  pipeline.tf_record_iterator(input_dir, pipeline_instance.input_type),
  output_dir)

2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,772 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,780 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,780 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,780 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,780 - tensorflow - WARNING - Key signatures ignored by TranspositionPipeline.
2023-04-16 18:38:15,

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
from magenta.pipelines import pipeline, note_sequence_pipelines, statistics
from note_seq.protobuf import music_pb2
from magenta.models import polyphony_rnn
from magenta.models.polyphony_rnn import polyphony_rnn_pipeline
from magenta.models.polyphony_rnn import polyphony_model
pipeline_instance = polyphony_rnn.polyphony_rnn_pipeline.get_pipeline(
    min_steps=80,  # 5 measures
    max_steps = None,
    eval_ratio=0.2,
    config=polyphony_model.default_configs['polyphony'])

input_dir = '/kaggle/tmp/bach.tfrecord'
output_dir = '/kaggle/working/bachrecords'
pipeline.run_pipeline_serial(
  pipeline_instance,
  pipeline.tf_record_iterator(input_dir, pipeline_instance.input_type),
  output_dir)


In [12]:
print(sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/working/split/eval_poly_tracks.tfrecord')),
sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/working/split/training_poly_tracks.tfrecord')))

1125 4752
